# Lab 5


Matrix Representation: In this lab you will be creating a simple linear algebra system. In memory, we will represent matrices as nested python lists as we have done in lecture. In the exercises below, you are required to explicitly test every feature you implement, demonstrating it works.

1. Create a `matrix` class with the following properties:
    * It can be initialized in 2 ways:
        1. with arguments `n` and `m`, the size of the matrix. A newly instanciated matrix will contain all zeros.
        2. with a list of lists of values. Note that since we are using lists of lists to implement matrices, it is possible that not all rows have the same number of columns. Test explicitly that the matrix is properly specified.
    * Matrix instances `M` can be indexed with `M[i][j]` and `M[i,j]`.
    * Matrix assignment works in 2 ways:
        1. If `M_1` and `M_2` are `matrix` instances `M_1=M_2` sets the values of `M_1` to those of `M_2`, if they are the same size. Error otherwise.
        2. In example above `M_2` can be a list of lists of correct size.


2. Add the following methods:
    * `shape()`: returns a tuple `(n,m)` of the shape of the matrix.
    * `transpose()`: returns a new matrix instance which is the transpose of the matrix.
    * `row(n)` and `column(n)`: that return the nth row or column of the matrix M as a new appropriately shaped matrix object.
    * `to_list()`: which returns the matrix as a list of lists.
    *  `block(n_0,n_1,m_0,m_1)` that returns a smaller matrix located at the n_0 to n_1 columns and m_0 to m_1 rows. 
    * (Extra credit) Modify `__getitem__` implemented above to support slicing.
        

3. Write functions that create special matrices (note these are standalone functions, not member functions of your `matrix` class):
    * `constant(n,m,c)`: returns a `n` by `m` matrix filled with floats of value `c`.
    * `zeros(n,m)` and `ones(n,m)`: return `n` by `m` matrices filled with floats of value `0` and `1`, respectively.
    * `eye(n)`: returns the n by n identity matrix.

4. Add the following member functions to your class. Make sure to appropriately test the dimensions of the matrices to make sure the operations are correct.
    * `M.scalarmul(c)`: a matrix that is scalar product $cM$, where every element of $M$ is multiplied by $c$.
    * `M.add(N)`: adds two matrices $M$ and $N$. Don’t forget to test that the sizes of the matrices are compatible for this and all other operations.
    * `M.sub(N)`: subtracts two matrices $M$ and $N$.
    * `M.mat_mult(N)`: returns a matrix that is the matrix product of two matrices $M$ and $N$.
    * `M.element_mult(N)`: returns a matrix that is the element-wise product of two matrices $M$ and $N$.
    * `M.equals(N)`: returns true/false if $M==N$.

5. Overload python operators to appropriately use your functions in 4 and allow expressions like:
    * 2*M
    * M*2
    * M+N
    * M-N
    * M*N
    * M==N
    * M=N


6. Demonstrate the basic properties of matrices with your matrix class by creating two 2 by 2 example matrices using your Matrix class and illustrating the following:

$$
(AB)C=A(BC)
$$
$$
A(B+C)=AB+AC
$$
$$
AB\neq BA
$$
$$
AI=A
$$

In [1]:
class Matrix:
    def __init__(self, n, m=None, values=None):
        if values is not None:
            self.data = values
            self.n = len(values)
            self.m = len(values[0]) if values else 0
            for row in values:
                if len(row) != self.m:
                    raise ValueError("All rows must have the same number of columns")
        else:
            if m is None:
                raise ValueError("Must specify both dimensions or provide a list of lists")
            self.n, self.m = n, m
            self.data = [[0] * m for _ in range(n)]

    def __getitem__(self, key):
        if isinstance(key, tuple):
            return self.data[key[0]][key[1]]
        return self.data[key]

    def __setitem__(self, key, value):
        if isinstance(key, tuple):
            self.data[key[0]][key[1]] = value
        else:
            self.data[key] = value

    def shape(self):
        return self.n, self.m

    def transpose(self):
        return Matrix(self.m, self.n, [[self.data[j][i] for j in range(self.n)] for i in range(self.m)])

    def row(self, n):
        return Matrix(1, self.m, [self.data[n]])

    def column(self, n):
        return Matrix(self.n, 1, [[self.data[i][n]] for i in range(self.n)])

    def to_list(self):
        return self.data

    def block(self, n_0, n_1, m_0, m_1):
        return Matrix(n_1 - n_0, m_1 - m_0, [row[m_0:m_1] for row in self.data[n_0:n_1]])

    def scalarmul(self, c):
        return Matrix(self.n, self.m, [[c * self.data[i][j] for j in range(self.m)] for i in range(self.n)])

    def add(self, N):
        if self.shape() != N.shape():
            raise ValueError("Matrix dimensions must match for addition")
        return Matrix(self.n, self.m, [[self.data[i][j] + N[i, j] for j in range(self.m)] for i in range(self.n)])

    def sub(self, N):
        if self.shape() != N.shape():
            raise ValueError("Matrix dimensions must match for subtraction")
        return Matrix(self.n, self.m, [[self.data[i][j] - N[i, j] for j in range(self.m)] for i in range(self.n)])

    def mat_mult(self, N):
        if self.m != N.n:
            raise ValueError("Matrix dimensions must be compatible for multiplication")
        return Matrix(self.n, N.m, [[sum(self.data[i][k] * N[k, j] for k in range(self.m)) for j in range(N.m)] for i in range(self.n)])

    def element_mult(self, N):
        if self.shape() != N.shape():
            raise ValueError("Matrix dimensions must match for element-wise multiplication")
        return Matrix(self.n, self.m, [[self.data[i][j] * N[i, j] for j in range(self.m)] for i in range(self.n)])

    def equals(self, N):
        return self.data == N.to_list()

    def __mul__(self, other):
        if isinstance(other, (int, float)):
            return self.scalarmul(other)
        return self.mat_mult(other)

    def __rmul__(self, other):
        return self.__mul__(other)

    def __add__(self, other):
        return self.add(other)

    def __sub__(self, other):
        return self.sub(other)

    def __eq__(self, other):
        return self.equals(other)

    @staticmethod
    def constant(n, m, c):
        return Matrix(n, m, [[c] * m for _ in range(n)])

    @staticmethod
    def zeros(n, m):
        return Matrix(n, m)

    @staticmethod
    def ones(n, m):
        return Matrix(n, m, [[1] * m for _ in range(n)])

    @staticmethod
    def eye(n):
        return Matrix(n, n, [[1 if i == j else 0 for j in range(n)] for i in range(n)])

# Demonstration of matrix properties
A = Matrix(2, 2, [[1, 2], [3, 4]])
B = Matrix(2, 2, [[5, 6], [7, 8]])
C = Matrix(2, 2, [[2, 0], [1, 2]])
I = Matrix.eye(2)

# (AB)C = A(BC)
assert (A.mat_mult(B)).mat_mult(C) == A.mat_mult(B.mat_mult(C))

# A(B + C) = AB + AC
assert A.mat_mult(B.add(C)) == A.mat_mult(B).add(A.mat_mult(C))

# AB != BA
assert A.mat_mult(B) != B.mat_mult(A)

# AI = A
assert A.mat_mult(I) == A

In [3]:
# 1. Creating matrices
print("Creating Matrices")
A = Matrix(2, 2, [[1, 2], [3, 4]])
B = Matrix(2, 2, [[5, 6], [7, 8]])
print("Matrix A:", A.to_list())  # [[1, 2], [3, 4]]
print("Matrix B:", B.to_list())  # [[5, 6], [7, 8]]

# 2. Shape
print("\nShape of A:", A.shape())  # (2, 2)

# 3. Transpose
print("\nTranspose of A:", A.transpose().to_list())  
# [[1, 3], [2, 4]]

# 4. Row and Column Extraction
print("\nRow 1 of A:", A.row(1).to_list())  # [[3, 4]]
print("Column 0 of A:", A.column(0).to_list())  # [[1], [3]]

# 5. Block Extraction
print("\nBlock of A (0,2,0,1):", A.block(0, 2, 0, 1).to_list())  
# [[1], [3]]

# 6. Scalar Multiplication
print("\nA * 2:", (A * 2).to_list())  
# [[2, 4], [6, 8]]

# 7. Addition
print("\nA + B:", (A + B).to_list())  
# [[6, 8], [10, 12]]

# 8. Subtraction
print("\nA - B:", (A - B).to_list())  
# [[-4, -4], [-4, -4]]

# 9. Matrix Multiplication
print("\nA * B:", (A * B).to_list())  
# [[19, 22], [43, 50]]

# 10. Element-wise Multiplication
print("\nElement-wise A * B:", A.element_mult(B).to_list())  
# [[5, 12], [21, 32]]

# 11. Identity Matrix
I = Matrix.eye(2)
print("\nIdentity Matrix:", I.to_list())  
# [[1, 0], [0, 1]]
print("A * I:", (A * I).to_list())  
# [[1, 2], [3, 4]] (A remains unchanged)

# 12. Equality
print("\nIs A equal to A?", A == A)  # True
print("Is A equal to B?", A == B)  # False

# 13. Matrix Properties
# (AB)C = A(BC)
C = Matrix(2, 2, [[2, 0], [1, 2]])
AB = A * B
BC = B * C
print("\n(AB)C:", (AB * C).to_list())  
print("A(BC):", (A * BC).to_list())  
# They should be equal

# A(B + C) = AB + AC
B_plus_C = B + C
AB_plus_AC = (A * B) + (A * C)
print("\nA(B+C):", (A * B_plus_C).to_list())  
print("AB + AC:", AB_plus_AC.to_list())  
# They should be equal

# AB != BA
print("\nAB:", AB.to_list())  
print("BA:", (B * A).to_list())  
# Should be different

# AI = A
print("\nA * I:", (A * I).to_list())  
# Should be same as A


Creating Matrices
Matrix A: [[1, 2], [3, 4]]
Matrix B: [[5, 6], [7, 8]]

Shape of A: (2, 2)

Transpose of A: [[1, 3], [2, 4]]

Row 1 of A: [[3, 4]]
Column 0 of A: [[1], [3]]

Block of A (0,2,0,1): [[1], [3]]

A * 2: [[2, 4], [6, 8]]

A + B: [[6, 8], [10, 12]]

A - B: [[-4, -4], [-4, -4]]

A * B: [[19, 22], [43, 50]]

Element-wise A * B: [[5, 12], [21, 32]]

Identity Matrix: [[1, 0], [0, 1]]
A * I: [[1, 2], [3, 4]]

Is A equal to A? True
Is A equal to B? False

(AB)C: [[60, 44], [136, 100]]
A(BC): [[60, 44], [136, 100]]

A(B+C): [[23, 26], [53, 58]]
AB + AC: [[23, 26], [53, 58]]

AB: [[19, 22], [43, 50]]
BA: [[23, 34], [31, 46]]

A * I: [[1, 2], [3, 4]]
